<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_dataput_data

In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)

W0804 12:02:27.535402 17356 deprecation.py:323] From <ipython-input-2-410b479fd05e>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0804 12:02:27.537317 17356 deprecation.py:323] From d:\pl\python\python37\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0804 12:02:27.538807 17356 deprecation.py:323] From d:\pl\python\python37\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use t

Extracting data/train-images-idx3-ubyte.gz


W0804 12:02:28.530342 17356 deprecation.py:323] From d:\pl\python\python37\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0804 12:02:28.656822 17356 deprecation.py:323] From d:\pl\python\python37\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz


W0804 12:02:28.809092 17356 deprecation.py:323] From d:\pl\python\python37\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting data/t10k-labels-idx1-ubyte.gz


In [14]:
# placeholder 정의
x = tf.placeholder('float', shape=[None, 784])
y_true = tf.placeholder('float', shape=[None, 10])

x_img = tf.reshape(x, [-1, 28, 28, 1])  # 784 = 28 * 28
x_img.get_shape()

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(1)])

* 합성곱 계층
    * feature map의 집합을 말한다.
    * 각 feature map(kernel, filter)는 하나의 특징을 검출한다.
    * cnn은 입력 계층의 일부 데이터만 은닉 계층에 연결
    * w, b(kernel, filter, feature map) 를 모든 은닉계층이 공유 => 가중치 매개 변수 감소
    * 5*5 window 사용:  28 * 28 => 24 * 24(28-5+1)
    * stride: 윈도우가 한번에 움직이는 거리
    * padding: 테두리의 크기
* 풀링 계층
    * 데이터 압축
    * ex: max pooling은 2x2 영역에서 가장 큰 값을 선택해 정보를 압축한다.
    * 특징의 정확한 위치보다는 다른 특징과의 상대적 위치가 중요함.
* 차원 변화
    * 입력 계층(28x28)
    * 합성곱 계층(None x 24 x 24, window=5, stride=1)
    * 풀링 계층(None x 12 x 12, maxpulling) 


In [6]:
def weight_variable(shape):
    # w 초기화
    # truncated normal: 정규분포를 따르는 수열 생성, 2sigma 이상인 값은 제외
    init = tf.truncated_normal(shape, stddev=.1)
    return tf.Variable(init)


def bias_variable(shape):
    # b 초기화, 0.1
    init = tf.constant(.1, shape=shape)
    return tf.Variable(init)


def conv2d(x, w):
    # 합성곱 계층
    # stride: The stride of the sliding window for each dimension of input
    return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    # ksize: 커널 크기 2x2
    # strides: 2x2
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [15]:
## layer 1
# window: 5x5, 32개 필터 정의
# 세번째 원소는 color
w_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_img, w_conv1) + b_conv1) # relu
h_pool1 = max_pool_2x2(h_conv1)  # pooling

## layer2
# 32: conv1에서 출력 크기
# window: 5x5, 64개 필터 정의
w_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


## output
# 1024뉴런
w_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])  # 직렬화
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder('float')  # 드롭아웃 되지않을 확률
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# softmax
w_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])  # 0-10
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

In [23]:
## train model

cross_entropy = -tf.reduce_sum(y_true * tf.log(y_conv))  # loss function
train_step = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cross_entropy)  # optimizer
correct_pred = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_true, 1))  # y_true is one-hot encoded
accuracy = tf.reduce_mean(tf.cast(correct_pred, 'float'))  # metric: accuracy

sess = tf.Session()
sess.run(tf.global_variables_initializer())  # 변수 초기화

for i in range(1000):
    batch = mnist.train.next_batch(100)  # 100개씩 배치학습
    if i%100 == 0:
        # 100번마다 정확도 출력
        train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y_true: batch[1], keep_prob: 1})
        print(f' iter {i} training accuracy: {train_accuracy}')
    sess.run(train_step, feed_dict={x: batch[0], y_true: batch[1], keep_prob: 0.5})
print('end')

 iter 0 training accuracy: 0.10000000149011612
 iter 100 training accuracy: 0.8399999737739563
 iter 200 training accuracy: 0.9399999976158142
 iter 300 training accuracy: 0.9200000166893005
 iter 400 training accuracy: 0.9300000071525574
 iter 500 training accuracy: 0.9700000286102295
 iter 600 training accuracy: 0.9700000286102295
 iter 700 training accuracy: 0.9300000071525574
 iter 800 training accuracy: 0.9399999976158142
 iter 900 training accuracy: 0.9800000190734863
end


In [24]:
test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1})
print(f'test accuracy: {test_accuracy}')

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_8' with dtype float and shape [?,10]
	 [[node Placeholder_8 (defined at <ipython-input-14-2761b4ebef96>:3) ]]

Original stack trace for 'Placeholder_8':
  File "d:\pl\python\python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\pl\python\python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\pl\python\python37\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\pl\python\python37\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "d:\pl\python\python37\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "d:\pl\python\python37\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "d:\pl\python\python37\lib\asyncio\base_events.py", line 523,